In [28]:
#Instalando a biblioteca no Google Gemin
!pip install -q -U google-generativeai

#Importando bibliotecas do Google Gemin, mumpy e pandas
import numpy as np
import pandas as pd
import google.generativeai as gemini

#Obtendo dados do usuário
from google.colab import userdata

#Armazenando a chave (API_KEY) em uma variável do colab
api_key = userdata.get('SECRET_KEY')

#Atribuindo a chave às configurações do gemini
gemini.configure(api_key=api_key)

for m in gemini.list_models():
	if 'embedContent' in m.supported_generation_methods:
		print(m.name)

#Gerando o primeiro Embed (Contexto)
text = "Hello world"
result = gemini.embed_content(model="models/embedding-001", content=text)
print(result['embedding'])
print(len(result['embedding']))

#Gerando o segundo Embed (Contexto)
text = [
    "What is the meaning of life?",
    "How much wood would a woodchuck chuck?",
    "How does the brain work?"]
result = gemini.embed_content(model="models/embedding-001", content=text)
for embedding in result['embedding']:
  print(str(embedding)[:50], "...TRIMMED")

#Gerando o terceiro Embed (Contexto)
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemin API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativas")

embedding = gemini.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")
print(embedding)

#Listagem de documentos
document1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar"}

document2 = {
    "Título": "TouchScreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso à internet"}

document3 = {
    "Título": "Mudança de marcha",
    "Conteúdo": "O Googlecar tem um transmissão automática. Para trocar as marchas."}

documents = [document1, document2, document3]

df = pd.DataFrame(documents)
df.columns = ['Titulo', 'Conteudo']

model = "models/embedding-001"

def embed_fn(title, text):
  return gemini.embed_content(model=model, content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

#Gerando função para retornar conteudo do Embedding criando anteriormente
def consultar_documento(consulta, base, model):
  embedding = gemini.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding)
  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]["Conteudo"]

consulta = "Como trocar marcha?"
resultado = consultar_documento(consulta, df, model)
print(resultado)

#Mesclando o modelo criado acima (embaddings) com o modelo existente (gemini)
prompt = f"Reescreva esse texto de uma forma mais criativa: {resultado}"
generation_conf = {
    "temperature": 0.5,
    "candidate_count": 1,
}
model2 = gemini.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_conf)
response = model2.generate_content(prompt)
print(response.text)


models/embedding-001
models/text-embedding-004
[0.04703258, -0.040190056, -0.029026963, -0.026809642, 0.018920582, -8.3654784e-05, 0.031116402, -0.019520544, 0.0114913415, 0.009625779, 0.04571186, 0.05170951, -0.007854084, -0.07627559, -0.00073652336, -0.02259244, 0.01149677, -0.00761096, 0.006400746, -0.0036826304, -8.6395165e-05, 0.007910556, -0.031401973, -0.027668774, 0.0131483, 0.005762955, -0.0022430476, -0.07029421, 0.007011013, 0.07013052, -0.047634568, 0.008311825, -0.060211696, 0.016431302, 0.042709153, -0.047674265, 0.03426082, 0.021967327, -0.0070651034, 0.00032590108, 0.013825696, -0.08921293, -0.03404069, -0.03793646, 0.059349738, -0.0044174152, 0.015472682, -0.0061533544, 0.022183485, -0.08739371, 0.049185753, 0.025158774, 0.044854913, -0.022910612, 0.02060697, -0.016286727, 0.07367813, 0.013565082, -0.06963922, -0.002877564, 0.02369202, 0.0143784685, -0.012660949, 0.06607742, -0.00069232617, -0.017637717, -0.06946077, 0.042905096, 0.03502765, -0.029362002, 0.0069921436,

[0.04703258, -0.040190056, -0.029026963, -0.026809642, 0.018920582, -8.3654784e-05, 0.031116402, -0.019520544, 0.0114913415, 0.009625779, 0.04571186, 0.05170951, -0.007854084, -0.07627559, -0.00073652336, -0.02259244, 0.01149677, -0.00761096, 0.006400746, -0.0036826304, -8.6395165e-05, 0.007910556, -0.031401973, -0.027668774, 0.0131483, 0.005762955, -0.0022430476, -0.07029421, 0.007011013, 0.07013052, -0.047634568, 0.008311825, -0.060211696, 0.016431302, 0.042709153, -0.047674265, 0.03426082, 0.021967327, -0.0070651034, 0.00032590108, 0.013825696, -0.08921293, -0.03404069, -0.03793646, 0.059349738, -0.0044174152, 0.015472682, -0.0061533544, 0.022183485, -0.08739371, 0.049185753, 0.025158774, 0.044854913, -0.022910612, 0.02060697, -0.016286727, 0.07367813, 0.013565082, -0.06963922, -0.002877564, 0.02369202, 0.0143784685, -0.012660949, 0.06607742, -0.00069232617, -0.017637717, -0.06946077, 0.042905096, 0.03502765, -0.029362002, 0.0069921436, -0.03341513, 0.036520302, -0.039816536, -0.025